# Imports

Import requests and bs4 for webscraping, pandas for dataframe manipulation, google drive for file saving

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Web Scraping Links

In [ ]:
url='https://thespruce.com/plants-a-to-z-5116344'
response = requests.get(url)

In [ ]:
html = response.content
soup = bs(html, "lxml")
print(soup)

In [ ]:
#Create a list of all elements in the html code that have this class and id
#Basically, make a list of links for each plant
container = soup("script", class_="comp schema-unified mntl-schema-unified", id="schema-unified_1-0")
container

# Clean Links DataFrame

Get rid of unnecessary characters to just keep the raw URLs

In [ ]:
for item in container:
  urls = str(item).split(sep="url")

df = pd.DataFrame(columns = [['URL']])
df['URL'] = urls
df = df[1:1271]
df = df.reset_index(drop = True)

df = df.replace(r'^.+://', 'http://', regex=True)

df

,URL
0,http://www.thespruce.com/abelia-growing-guide-...
1,http://www.thespruce.com/growing-acoma-crape-m...
2,http://www.thespruce.com/growing-aeonium-plant...
3,http://www.thespruce.com/grow-aerangis-orchids...
4,http://www.thespruce.com/growing-and-caring-fo...
...,...
1265,http://www.thespruce.com/zebra-grass-informati...
1266,http://www.thespruce.com/grow-zebra-plants-ins...
1267,http://www.thespruce.com/growing-devils-backbo...
1268,http://www.thespruce.com/zinfin-doll-hydrangea...


In [ ]:
df = df.to_csv('URLs.csv')

In [ ]:
plantLinks = pd.read_csv('URLs.csv')
del plantLinks['Unnamed: 0']
plantLinks

,URL
0,http://www.thespruce.com/abelia-growing-guide-...
1,http://www.thespruce.com/growing-acoma-crape-m...
2,http://www.thespruce.com/growing-aeonium-plant...
3,http://www.thespruce.com/grow-aerangis-orchids...
4,http://www.thespruce.com/growing-and-caring-fo...
...,...
1265,http://www.thespruce.com/zebra-grass-informati...
1266,http://www.thespruce.com/grow-zebra-plants-ins...
1267,http://www.thespruce.com/growing-devils-backbo...
1268,http://www.thespruce.com/zinfin-doll-hydrangea...


In [ ]:
urls = []
for plant in plantLinks.URL:
  url = plant[:-44]
  urls.append(url)

plantLinks.URL = urls
plantLinks.URL = plantLinks.URL.str.replace("(\").*","")
plantLinks.URL = plantLinks.URL.str.replace("(}).*","")
plantLinks.URL = plantLinks.URL.str.replace("\n","")
plantLinks.to_csv('/content/gdrive/MyDrive/Tamid Group/Let it Bloom/Data/Plant_List.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


# Webscrape Plant Data

Read in dataframe of URLs

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Tamid Group/Let it Bloom/Data/Plant_List.csv')
del df['Unnamed: 0']
df

,URL
0,http://www.thespruce.com/abelia-growing-guide-...
1,http://www.thespruce.com/growing-acoma-crape-m...
2,http://www.thespruce.com/growing-aeonium-plant...
3,http://www.thespruce.com/grow-aerangis-orchids...
4,http://www.thespruce.com/growing-and-caring-fo...
...,...
1265,http://www.thespruce.com/zebra-grass-informati...
1266,http://www.thespruce.com/grow-zebra-plants-ins...
1267,http://www.thespruce.com/growing-devils-backbo...
1268,http://www.thespruce.com/zinfin-doll-hydrangea...


In [ ]:
#Function that takes in a URL and creates a row of a dataframe with all of the data

In [ ]:
def getPlantData(url):
  response = requests.get(url)
  html = response.content
  soup = bs(html, "lxml")

  table = soup.find('table', class_='mntl-sc-block-table__table')

  try:
   # Obtain every title of columns with tag <b>
    headers = []
    for i in table.find_all('b'):
      title = i.text
      headers.append(title)
  
    df2 = pd.DataFrame(columns = headers)

    # Create a for loop to fill data
    count = 0
    for j in table.find_all('tr')[0:]:

      header = j.find('b').text
      length = len(df2)
      if header == headers[count]:
       df2[header] = [j.text.replace("\n"+header+"\n","").replace("\n", "").replace(header,"")]
       count+=1
  except:
    df2 = pd.DataFrame(columns = headers)

  return df2

In [ ]:
getPlantData(df.URL[10])

,Common Name,Botanical Name,Family,Plant Type,Mature Size,Sun Exposure,Soil Type,Soil pH,Bloom Time,Flower Color,Hardiness Zones,Native Area,Toxicity
0,"Agave, century plant",Agave,Asparagaceae,"Perennial, succulent","1–20 ft. tall, 1–10 ft. wide (depends on variety)",Full,"Sandy, well-drained","Acidic, neutral",Varies — most plants only bloom once in their ...,"Green, white, yellow","5–11, USA","North America, Central America, South America","Toxic to people, toxic to pets"


In [ ]:
#Run through every row and add it to a new dataframe
finaldf = getPlantData(df.URL[0])

i = 1
while (i < len(df)):
  try:
    print(getPlantData(df.URL[i]))
    finaldf = finaldf.append(getPlantData(df.URL[i]))
    finaldf = finaldf.reset_index(drop=True)
    i+=1
  except:
    i+=1
finaldf

In [ ]:
finaldf.to_csv('/content/gdrive/MyDrive/Tamid Group/Let it Bloom/Data/Final_Plant_List.csv')

# Clean Data

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Tamid Group/Let it Bloom/Data/Final_Plant_List.csv')
#Remove unneeded columns
df = df[['Common Name', 'Botanical Name', 'Family', 'Plant Type', 'Mature Size', 
         'Sun Exposure', 'Soil Type', 'Soil pH', 'Bloom Time', 'Flower Color',
         'Hardiness Zones', 'Native Area', 'Toxicity', 'Native Areas', 'USDA Hardiness Zones']]
#Fill in NA values
df = df.fillna('')
#Combine repeat columns
df['Native Area'] = df['Native Area'].astype(str) + df['Native Areas']
del df['Native Areas']
df['Hardiness Zones'] = df['Hardiness Zones'].astype(str) + df['USDA Hardiness Zones']
del df['USDA Hardiness Zones']

df = df.to_csv('/content/gdrive/MyDrive/Tamid Group/Let it Bloom/Data/Final_Plant_List.csv')

In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/Tamid Group/Let it Bloom/Data/Final_Plant_List.csv')
del df['Unnamed: 0']
df

,Common Name,Botanical Name,Family,Plant Type,Mature Size,Sun Exposure,Soil Type,Soil pH,Bloom Time,Flower Color,Hardiness Zones,Native Area,Toxicity
0,Abelia,Abelia,Caprifoliaceae,Shrub,"2-10 ft. tall, 2-8 ft. wide","Full, Partial","Loamy, Moist but well-drained","Acidic, Neutral","Spring, Summer, Fall","Pink, Yellow, White","4-11, USA",Asia,NaN
1,Acoma crape myrtle,Lagerstroemia x ‘Acoma’,Lythraceae,Tree,2-15 ft. tall and 2-10 ft. wide,Full,Well-draining,Slightly acidic to neutral,"Spring, summer",White,"7, 8, 9","Asia, Australia",NaN
2,Aeonium,Aeonium spp.,Crassulaceae,Perennial succulent,2–60 in. (depending on species and variety),Full sun to part shade,Sandy loam,5.6–6.0 (slightly acidic),Late winter or spring,"Pink (flowering is rare, occurring only in mat...",9–11 (USDA); often grown as potted plants,"Canary Islands, Africa",NaN
3,Aerangis orchids,Aerangis spp.,NaN,Epiphytic orchids,6–24 inches (varies by species),"Part sun, bright filtered light",Orchid potting mix,5.5 to 6.0 (acidic),Spring to fall (varies by species),"White, yellow",10–11 (USDA),"Tropical Africa, Madagascar",NaN
4,Osteospermum spp.,"African daisy, cape daisy, osteospermum",Asteraceae,Perennial,"1–3 ft. tall, 1–2 ft. wide",Full sun,"Moist, well-drained",5.0 -5.5 (Acidic),"Spring, summer, fall","Purple, pink, yellow, orange, white, bicolor",10–11 (USDA),"Africa, Southwestern Asia",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,Yucca plant,Yucca spp.,Agavaceae,Herbaceous perennials,Varies by variety,"Bright, indirect light","Sandy, well-draining",Neutral to acidic,"Mid-summer, early fall (rarely flowers indoors)","Creamy white, pink","9 to 11, USA","North America, the Caribbean",Toxic to pets
1033,Zebra plant,Aphelandra squarrosa,Acanthaceae,Annual,"1–2 ft. tall (indoors), 4–6 ft. tall (outdoors...","Indirect, partial",Moist,Neutral to acidic,"Late summer, early fall",Yellow,"11, 12 (USDA)",Brazil,NaN
1034,"Zigzag plant, red bird flower, fiddle flower",Euphorbia tithymaloides (formerly Pedilanthus ...,NaN,Tropical succulent,2-3 ft. tall,"Bright, indirect sun","Rich, well-drained, slightly acidic",6.1- 7.8,Summer,"White, pink, red",9-10 (USDA),Tropical and sub-tropical North and Central Am...,Sap is toxic to humans and animals
1035,Zinfin Doll hydrangea,Hydrangea paniculata 'Zinfin Doll',Hydrangeaceae,Shrub,"4-6 ft. tall, and 4-6 ft. wide","Full, partial",Well-drained,Neutral to acidic,Summer,"White, pink, red",NaN,NaN,NaN
